# 📒 Notebook d'évaluation du modèle (version Colab)

Ce dernier a pour objectif de vous aider à **vérifier l'exportation correcte de votre modèle au format ONNX** et à valider la bonne généralisation de votre modèle. Pour l'évaluation finale, un autre jeu de données sera utilisé.

⚠️ Ce notebook doit fonctionner sans aucune modification du code de ce notebook. Si une erreur survient, vérifiez attentivement la procédure utilisée pour convertir votre modèle au format ONNX.

## 🛠️ Installation et importation des bibliothèques Python

In [1]:
!pip install onnxruntime

import urllib.request
import onnxruntime as ort
import numpy as np
from sklearn.metrics import accuracy_score, classification_report
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder
import ipywidgets as widgets
from IPython.display import display
from google.colab import files

In [2]:
def to_categorical(y, num_classes=None, dtype='float32'):
    """
    Convertit un vecteur de classes en une matrice d'encodage one-hot.

    Args:
        y (array-like): Vecteur d'étiquettes (entiers).
        num_classes (int): Nombre total de classes. Si None, sera déterminé automatiquement.
        dtype (str): Type des données de sortie (par défaut 'float32').

    Returns:
        numpy.ndarray: Matrice d'encodage one-hot.
    """
    y = np.array(y, dtype='int')
    if not num_classes:
        num_classes = np.max(y) + 1
    one_hot = np.zeros((y.shape[0], num_classes), dtype=dtype)
    one_hot[np.arange(y.shape[0]), y] = 1
    return one_hot

## Téléchargement du jeu de données d'évaluation

In [3]:
file_url = "https://gricad-gitlab.univ-grenoble-alpes.fr/phelma-sicom/2a/4pmsiia4-td/-/raw/main/content/3_mini-projet/dataset_evaluation.npz?ref_type=heads&inline=false"
local_filename = "dataset_evaluation.npz"
urllib.request.urlretrieve(file_url, local_filename)
print(f"Fichier téléchargé : {local_filename}")

Fichier téléchargé : dataset_evaluation.npz


## Téléchargement de votre modèle entrainé

Votre modèle doit être chargé au format **ONNX**.

In [4]:
uploaded = files.upload()
for file_name, content in uploaded.items():
    print(f"Fichier chargé : {file_name}")
    try:
        session = ort.InferenceSession(content)
        print("Session ONNX Runtime créée avec succès !")
        input_names = [inp.name for inp in session.get_inputs()]
        output_names = [out.name for out in session.get_outputs()]
        print(f"Entrées : {input_names}")
        print(f"Sorties : {output_names}")
    except Exception as e:
        print(f"Erreur lors de la création de la session ONNX Runtime : {e}")

Saving model_lr.onnx to model_lr.onnx
Fichier chargé : model_lr.onnx
Session ONNX Runtime créée avec succès !
Entrées : ['float_input']
Sorties : ['output_label', 'output_probability']


## Test de votre modèle entrainé

In [5]:
labels = [
    'Tente', 'Sac à dos', 'Piano', 'Pingouin', 'Dragon',
    'Vélo', 'Bonhomme de neige', 'Flocon de neige', 'Ordinateur portable', 'Montagne'
]
data = np.load("dataset_evaluation.npz")
X = data['images']
y_true = data['labels']
label_encoder = LabelEncoder()
label_encoder.fit(labels)
input_name = session.get_inputs()[0].name
try:
    y_pred = session.run(None, {input_name: X.reshape(10000, -1).astype(np.float32)})[0]
except:
    pred = np.array(session.run(None, {input_name: X.astype(np.float32)}))
    predicted_label_index = pred[0].argmax(axis=1)
    y_pred = [labels[int(i)] for i in predicted_label_index]
    y_true = label_encoder.transform(y_true)
    y_true = to_categorical(y_true, num_classes=len(labels))
    y_true = [labels[int(i)] for i in y_true.argmax(axis=1)]
report = classification_report(y_true, y_pred)
print(f"Justesse (accuracy) : {accuracy_score(y_true, y_pred)}")
print(report)

Justesse (accuracy) : 0.6823
                     precision    recall  f1-score   support

  Bonhomme de neige       0.85      0.80      0.82      1000
             Dragon       0.45      0.59      0.51      1000
    Flocon de neige       0.73      0.72      0.73      1000
           Montagne       0.64      0.78      0.70      1000
Ordinateur portable       0.68      0.80      0.73      1000
              Piano       0.84      0.19      0.31      1000
           Pingouin       0.76      0.79      0.77      1000
          Sac à dos       0.67      0.72      0.69      1000
              Tente       0.67      0.63      0.65      1000
               Vélo       0.76      0.80      0.78      1000

           accuracy                           0.68     10000
          macro avg       0.70      0.68      0.67     10000
       weighted avg       0.70      0.68      0.67     10000

